In [46]:
import numpy as np
import itertools
from skimage import color, io

In [50]:
def make_blocks_vectorized(x,d):
    return list(itertools.chain.from_iterable(np.hsplit(r, d) for r in np.split(x, d)))

In [74]:
S = 512 # original width and height of image
K = 64 # width and height of each piece

original_image = io.imread("../data/sources/1200.png")
original_squares = make_blocks_vectorized(original_image, S // K)
scrambled_image =  io.imread("../data/1200.png")
scrambled_squares = make_blocks_vectorized(scrambled_image, S // K)

# scrambled pieces
x_0 = color.rgb2lab(scrambled_squares[0])
x_1 = color.rgb2lab(scrambled_squares[1])

x_0p = color.rgb2lab(io.imread('../data/1200_00.png'))
x_1p = color.rgb2lab(io.imread('../data/1200_01.png'))

# correct pieces
x_2 = color.rgb2lab(original_squares[0])
x_3 = color.rgb2lab(original_squares[1])

In [75]:
x_0[0][63][0]

35.677094646907676

In [76]:
def dissimilarity(x_i, x_j, r='r'):
    return sum(sum((x_i[k][K-1][d] - x_j[k][0][d])**2 for d in range(3)) for k in range(K))

In [85]:
def vectorised_dissimilarity(x_i, x_j, r='r'):
    return np.sum(np.square(x_i[:,K-1] - x_j[:,0]))

In [86]:
dissimilarity(x_0, x_1)

50035.60969747558

In [87]:
vectorised_dissimilarity(x_0, x_1)

50035.60969747558

In [78]:
dissimilarity(x_2, x_3)

538.4051935376021

In [88]:
vectorised_dissimilarity(x_2, x_3)

538.4051935376021

In [79]:
def l_q_dissimiliarity(x_i, x_j, r='r', p=3/10, q=1/16):
    return np.power(sum(sum(np.power(np.absolute(x_i[k][K-1][d] - x_j[k][0][d]), p) for d in range(3)) for k in range(K)), q / p)

In [97]:
def vectorised_l_q_dissimiliarity(x_i, x_j, r='r', p=3/10, q=1/16):
    return np.power(np.sum(np.power(np.abs(x_i[:,K-1] - x_j[:,0]), p)), q/p)

In [98]:
l_q_dissimiliarity(x_0, x_1)

3.3933564063719466

In [99]:
vectorised_l_q_dissimiliarity(x_0, x_1)

3.3933564063719466

In [81]:
l_q_dissimiliarity(x_2, x_3)

2.929677477440781

In [100]:
vectorised_l_q_dissimiliarity(x_2, x_3)

2.929677477440781

In [103]:
def precalculate_dissimiliarities(D, parts, r='r'):
    dissimilarity_matrix = np.empty((len(parts), len(parts)))
    for i, x_i in enumerate(parts):
        for j, x_j in enumerate(parts):
            if i == j:
                continue
            dissimilarity_matrix[i][j] = D(x_i, x_j, r)
    return dissimilarity_matrix

In [106]:
precalculate_dissimiliarities(vectorised_l_q_dissimiliarity, scrambled_squares)

array([[ 0.00000000e+000,  3.76315622e+000,  4.15385359e+000, ...,
         4.19879441e+000,  3.83275961e+000,  4.09708726e+000],
       [ 3.94276481e+000,  3.60705816e-085,  4.15017941e+000, ...,
         4.14714063e+000,  3.78932799e+000,  4.07878260e+000],
       [ 3.98976968e+000,  4.05744808e+000,  6.01347002e-154, ...,
         3.95811111e+000,  3.99281101e+000,  3.98092906e+000],
       ...,
       [ 3.98102628e+000,  3.98646914e+000,  4.14115962e+000, ...,
        -6.85098432e+303,  3.80507569e+000,  4.13720391e+000],
       [ 4.19555309e+000,  3.93450356e+000,  4.12713219e+000, ...,
         4.17662826e+000,  1.41244276e-309,  4.16900490e+000],
       [ 3.96623656e+000,  3.84915895e+000,  4.18776420e+000, ...,
         3.80900165e+000,  3.74213063e+000, -5.31670221e+303]])